# Imports

In [38]:
import numpy as np
import pandas as pd
import datetime
import cvxpy as cp

pd.options.display.max_columns = 100

rawdata = pd.read_csv("data/elo/nfl_elo.csv")
rawdata.head()

,date,season,neutral,playoff,team1,team2,elo1_pre,elo2_pre,elo_prob1,elo_prob2,elo1_post,elo2_post,qbelo1_pre,qbelo2_pre,qb1,qb2,qb1_value_pre,qb2_value_pre,qb1_adj,qb2_adj,qbelo_prob1,qbelo_prob2,qb1_game_value,qb2_game_value,qb1_value_post,qb2_value_post,qbelo1_post,qbelo2_post,score1,score2
0,1920-09-26,1920,0,NaN,RII,STP,1503.947,1300.000,0.824651,0.175349,1516.108,1287.838,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,48.0,0.0
1,1920-10-03,1920,0,NaN,DAY,COL,1493.002,1504.908,0.575819,0.424181,1515.434,1482.475,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.0,0.0
2,1920-10-03,1920,0,NaN,RII,MUN,1516.108,1478.004,0.644171,0.355829,1542.135,1451.977,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45.0,0.0
3,1920-10-03,1920,0,NaN,CHI,MUT,1368.333,1300.000,0.682986,0.317014,1386.533,1281.800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0,0.0
4,1920-10-03,1920,0,NaN,CBD,PTQ,1504.688,1300.000,0.825267,0.174733,1516.803,1287.885,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,48.0,0.0


# Util Functions

In [39]:
#Functions

def get_tuesday(x):
    #Gets the tuesday preceeding each week's game- effectively the start of the week
    t = x['date'] + datetime.timedelta(days=(5-x['date'].weekday()))
    if x['date'].weekday()== 0:
        t = t - datetime.timedelta(days=7)
    return t



In [40]:
mydata = rawdata[rawdata['season'] == 2019]
mydata = mydata[mydata['playoff'].isna()]
mydata = mydata.reset_index()

keepcols = ['date','season','neutral','team1','team2','elo1_pre','elo2_pre','qbelo1_pre','qbelo2_pre',
            'elo_prob1','elo_prob2','qbelo_prob1','qbelo_prob2','score1','score2']
mydata = mydata[keepcols]

mydata['date'] = [datetime.datetime.strptime(d,'%Y-%m-%d') for d in mydata['date']]
#mydata['wday'] = [d.weekday() for d in mydata['date']]

mydata.head()
mydata.shape

(256, 15)

In [41]:
mydata['tuesday'] = mydata.apply(lambda x: get_tuesday(x),axis=1)
mydata['week'] = mydata.groupby('season')['tuesday'].rank("dense")
mydata['week'] = mydata['week'].astype('int')

mydata = mydata.drop('tuesday',axis=1)
mydata = mydata[['date','season','week','team1','team2','elo1_pre','elo2_pre',
                 'elo_prob1','elo_prob2','score1','score2']]
mydata.head()

,date,season,week,team1,team2,elo1_pre,elo2_pre,elo_prob1,elo_prob2,score1,score2
0,2019-09-05,2019,1,CHI,GB,1588.897931,1455.131370,0.758449,0.241551,3.0,10.0
1,2019-09-08,2019,1,CLE,TEN,1455.886401,1518.907204,0.502848,0.497152,13.0,43.0
2,2019-09-08,2019,1,PHI,WSH,1581.513559,1441.021792,0.765470,0.234530,32.0,27.0
3,2019-09-08,2019,1,CAR,LAR,1519.378772,1598.015666,0.480385,0.519615,27.0,30.0
4,2019-09-08,2019,1,JAX,KC,1455.249073,1602.077427,0.384370,0.615630,26.0,40.0


In [42]:
team1 = mydata[['week','team1','elo_prob1']]
team1 = team1.rename(columns={'team1':'team','elo_prob1':'wp'})
team2 = mydata[['week','team2','elo_prob2']]
team2 = team2.rename(columns={'team2':'team','elo_prob2':'wp'})

longdata = pd.concat([team1,team2]).drop_duplicates()
longdata.sort_values(['week','team'])
longdata.head()

,week,team,wp
0,1,CHI,0.758449
1,1,CLE,0.502848
2,1,PHI,0.765470
3,1,CAR,0.480385
4,1,JAX,0.384370


In [43]:
#Create mini-matrix for weeks 1 and 2 only, and only a few teams

mini = mydata[mydata.week.isin([1,2])]
mini = mini[(mini.team1.isin(['PHI','MIN','ATL'])) | (mini.team2.isin(['PHI','MIN','ATL']))]
mini['comboteam'] = mini['team1'] + mini['team2']
mini

,date,season,week,team1,team2,elo1_pre,elo2_pre,elo_prob1,elo_prob2,score1,score2,comboteam
2,2019-09-08,2019,1,PHI,WSH,1581.513559,1441.021792,0.765470,0.234530,32.0,27.0,PHIWSH
7,2019-09-08,2019,1,MIN,ATL,1538.423954,1520.315825,0.617372,0.382628,28.0,12.0,MINATL
23,2019-09-15,2019,2,GB,MIN,1489.807271,1559.316056,0.493512,0.506488,21.0,16.0,GBMIN
30,2019-09-15,2019,2,ATL,PHI,1499.423723,1589.200034,0.464404,0.535596,24.0,20.0,ATLPHI


In [44]:
#Brute force all combined win probabilities for three teams and two weeks PHI, ATL, MIN

team1 = mini[['week','team1','elo_prob1']]
team1 = team1.rename(columns={'team1':'team','elo_prob1':'wp'})
team2 = mini[['week','team2','elo_prob2']]
team2 = team2.rename(columns={'team2':'team','elo_prob2':'wp'})

teams = pd.concat([team1,team2]).drop_duplicates()
teams.sort_values(['week','team'])

teams

p = []
for t in ['PHI','ATL','MIN']:
    p1 = teams.loc[(teams.week==1) & (teams.team==t),'wp'].values[0]
    for t2 in ['PHI','ATL','MIN']:
        if t==t2:
            pass
        else:
            p2 = teams.loc[(teams.week==2) & (teams.team==t2),'wp'].values[0]
            p.append([t,t2,p1,p2,p1*p2])

pdf = pd.DataFrame(p,columns=['week1','week2','week1prob','week2prob','prob'])
pdf.sort_values('prob',ascending=False)

,week1,week2,week1prob,week2prob,prob
1,PHI,MIN,0.765470,0.506488,0.387702
0,PHI,ATL,0.765470,0.464404,0.355488
4,MIN,PHI,0.617372,0.535596,0.330662
5,MIN,ATL,0.617372,0.464404,0.286710
2,ATL,PHI,0.382628,0.535596,0.204934
3,ATL,MIN,0.382628,0.506488,0.193797


# Model 1 - Scalars, 3 teams, 2 weeks

In [45]:
#Scalar variable model

w1_phi = cp.Constant(teams[(teams.week==1) & (teams.team=='PHI')]['wp'].values[0])
w1_atl = cp.Constant(teams[(teams.week==1) & (teams.team=='ATL')]['wp'].values[0])
w1_min = cp.Constant(teams[(teams.week==1) & (teams.team=='MIN')]['wp'].values[0])
w2_phi = cp.Constant(teams[(teams.week==2) & (teams.team=='PHI')]['wp'].values[0])
w2_atl = cp.Constant(teams[(teams.week==2) & (teams.team=='ATL')]['wp'].values[0])
w2_min = cp.Constant(teams[(teams.week==2) & (teams.team=='MIN')]['wp'].values[0])

s1_phi = cp.Variable(boolean=True)
s1_atl = cp.Variable(boolean=True)
s1_min = cp.Variable(boolean=True)
s2_phi = cp.Variable(boolean=True)
s2_atl = cp.Variable(boolean=True)
s2_min = cp.Variable(boolean=True)

constraints = [s1_phi + s1_atl + s1_min == 1,
               s2_phi + s2_atl + s2_min == 1,
               s1_phi + s2_phi <= 1,
               s1_atl + s2_atl <= 1,
               s1_min + s2_min <= 1]

obj = cp.Maximize(w1_phi*s1_phi + w1_atl*s1_atl + w1_min*s1_min +
                  w2_phi*s2_phi + w2_atl*s2_atl + w2_atl*s2_min)

prob = cp.Problem(obj,constraints)

prob.solve()

print(np.round(s1_phi.value))
print(np.round(s1_atl.value))
print(np.round(s1_min.value))
print(np.round(s2_phi.value))
print(np.round(s2_atl.value))
print(np.round(s2_min.value))

1.0
0.0
0.0
0.0
0.0
1.0


# Model 2 - Vector-wise, 3 teams, 2 weeks

In [46]:
#Vector-wise model

#Teams in order: PHI, ATL, MIN
teams = ['PHI','MIN','ATL']
weeks = [1,2]

wts = []
selects = []
for wk in weeks:
    for t in teams:
        var = longdata[(longdata.week==wk) & (longdata.team==t)]['wp'].values[0]
        wts.append(var)
        print('Team: %s Week: %s WP: %s' % (t,wk,var))

        
weights = cp.Constant(wts)
selects = cp.Variable(6, boolean=True)

constraints = [sum(selects[range(0,3)]) == 1,
               sum(selects[range(3,6)]) == 1,
               selects[0] + selects[3] <= 1,
               selects[1] + selects[4] <= 1,
               selects[2] + selects[5] <= 1]

obj = cp.Maximize(np.multiply(weights,selects))

prob = cp.Problem(obj,constraints)

prob.solve()

#np.round(abs(selects.value))
for variable in prob.variables():
    print("Variable %s: value %s" % (variable.name(), np.round(variable.value)))

Team: PHI Week: 1 WP: 0.7654698543324963
Team: MIN Week: 1 WP: 0.6173720777976055
Team: ATL Week: 1 WP: 0.3826279222023945
Team: PHI Week: 2 WP: 0.5355956586211125
Team: MIN Week: 2 WP: 0.5064882997967771
Team: ATL Week: 2 WP: 0.4644043413788875
Variable var22405: value [1. 0. 0. 0. 1. 0.]


# Model 3 - Vector-wise, all teams

In [47]:
teams = np.unique(longdata.team)
teams.sort()
print(teams)
weeks = list(range(1,10))
print(weeks)

['ARI' 'ATL' 'BAL' 'BUF' 'CAR' 'CHI' 'CIN' 'CLE' 'DAL' 'DEN' 'DET' 'GB'
 'HOU' 'IND' 'JAX' 'KC' 'LAC' 'LAR' 'MIA' 'MIN' 'NE' 'NO' 'NYG' 'NYJ'
 'OAK' 'PHI' 'PIT' 'SEA' 'SF' 'TB' 'TEN' 'WSH']
[1, 2, 3, 4, 5, 6, 7, 8, 9]


In [48]:
wts = []
selects = []
english = []
var_mappings = []
for wk in weeks:
    for t in teams:
        try:
            var = longdata[(longdata.week==wk) & (longdata.team==t)]['wp'].values[0]
        except IndexError:
            var = -50
        
        wts.append(var)
        english.append('Team: %s Week: %s WP: %s' % (t,wk,var))
        var_mappings.append([wk,t,var])
        #print('Team: %s Week: %s WP: %s' % (t,wk,var))

weights = cp.Constant(wts)
selects = cp.Variable(len(teams)*len(weeks), boolean=True)
var_mappings = pd.DataFrame(var_mappings, columns=['week','team','prob'])
#[x + 1 for x in np.multiply(32,range(len(weeks)))]
#[x + 1 for x in range(32)]
#list(range(256,288))

In [49]:
constraints = [sum(selects[list(range(32))]) == 1, #Week 1
               sum(selects[list(range(32,64))]) == 1,
               sum(selects[list(range(64,96))]) == 1,
               sum(selects[list(range(96,128))]) == 1,
               sum(selects[list(range(128,160))]) == 1,
               sum(selects[list(range(160,192))]) == 1,
               sum(selects[list(range(192,224))]) == 1,
               sum(selects[list(range(224,256))]) == 1,
               sum(selects[list(range(256,288))]) == 1,
               sum(selects[[x + 0 for x in np.multiply(32,range(len(weeks)))]]) <= 1, #Team 1 all weeks
               sum(selects[[x + 1 for x in np.multiply(32,range(len(weeks)))]]) <= 1,
               sum(selects[[x + 2 for x in np.multiply(32,range(len(weeks)))]]) <= 1,
               sum(selects[[x + 3 for x in np.multiply(32,range(len(weeks)))]]) <= 1,
               sum(selects[[x + 4 for x in np.multiply(32,range(len(weeks)))]]) <= 1,
               sum(selects[[x + 5 for x in np.multiply(32,range(len(weeks)))]]) <= 1,
               sum(selects[[x + 6 for x in np.multiply(32,range(len(weeks)))]]) <= 1,
               sum(selects[[x + 7 for x in np.multiply(32,range(len(weeks)))]]) <= 1,
               sum(selects[[x + 8 for x in np.multiply(32,range(len(weeks)))]]) <= 1,
               sum(selects[[x + 9 for x in np.multiply(32,range(len(weeks)))]]) <= 1,
               sum(selects[[x + 10 for x in np.multiply(32,range(len(weeks)))]]) <= 1,
               sum(selects[[x + 11 for x in np.multiply(32,range(len(weeks)))]]) <= 1,
               sum(selects[[x + 12 for x in np.multiply(32,range(len(weeks)))]]) <= 1,
               sum(selects[[x + 13 for x in np.multiply(32,range(len(weeks)))]]) <= 1,
               sum(selects[[x + 14 for x in np.multiply(32,range(len(weeks)))]]) <= 1,
               sum(selects[[x + 15 for x in np.multiply(32,range(len(weeks)))]]) <= 1,
               sum(selects[[x + 16 for x in np.multiply(32,range(len(weeks)))]]) <= 1,
               sum(selects[[x + 17 for x in np.multiply(32,range(len(weeks)))]]) <= 1,
               sum(selects[[x + 18 for x in np.multiply(32,range(len(weeks)))]]) <= 1,
               sum(selects[[x + 19 for x in np.multiply(32,range(len(weeks)))]]) <= 1,
               sum(selects[[x + 20 for x in np.multiply(32,range(len(weeks)))]]) <= 1,
               sum(selects[[x + 21 for x in np.multiply(32,range(len(weeks)))]]) <= 1,
               sum(selects[[x + 22 for x in np.multiply(32,range(len(weeks)))]]) <= 1,
               sum(selects[[x + 23 for x in np.multiply(32,range(len(weeks)))]]) <= 1,
               sum(selects[[x + 24 for x in np.multiply(32,range(len(weeks)))]]) <= 1,
               sum(selects[[x + 25 for x in np.multiply(32,range(len(weeks)))]]) <= 1,
               sum(selects[[x + 26 for x in np.multiply(32,range(len(weeks)))]]) <= 1,
               sum(selects[[x + 27 for x in np.multiply(32,range(len(weeks)))]]) <= 1,
               sum(selects[[x + 28 for x in np.multiply(32,range(len(weeks)))]]) <= 1,
               sum(selects[[x + 29 for x in np.multiply(32,range(len(weeks)))]]) <= 1,
               sum(selects[[x + 30 for x in np.multiply(32,range(len(weeks)))]]) <= 1,
               sum(selects[[x + 31 for x in np.multiply(32,range(len(weeks)))]]) <= 1]

In [50]:
obj = cp.Maximize(np.multiply(weights,selects))

prob = cp.Problem(obj,constraints)

prob.solve()

results = pd.DataFrame(columns=['week','team','prob'])

for e,variable in enumerate(prob.variables()[0]):
    if np.round(variable.value)==1:
        print(english[e] + ' ' + str(np.round(variable.value)))
        results = results.append(var_mappings.iloc[e])
        
results

Team: CHI Week: 1 WP: 0.7584485193045968 1.0
Team: BAL Week: 2 WP: 0.8320536957330441 1.0
Team: DAL Week: 3 WP: 0.8292567833682122 1.0
Team: LAR Week: 4 WP: 0.8361690031716856 1.0
Team: PHI Week: 5 WP: 0.8482898424233134 1.0
Team: NE Week: 6 WP: 0.8911197569949159 1.0
Team: BUF Week: 7 WP: 0.8013633266223047 1.0
Team: NO Week: 8 WP: 0.8807481521184674 1.0
Team: SEA Week: 9 WP: 0.7546931445963736 1.0


,week,team,prob
5,1,CHI,0.758449
34,2,BAL,0.832054
72,3,DAL,0.829257
113,4,LAR,0.836169
153,5,PHI,0.848290
180,6,NE,0.891120
195,7,BUF,0.801363
245,8,NO,0.880748
283,9,SEA,0.754693


# Model 4 - Allow for team blocking, week eliminating

- Need to dynamically construct constraints
- Force team constraint to zero for teams that have been blocked
- Filter long game data to only future weeks
- Only include week constraints for the remaining # of weeks to optimize

In [51]:
teams = np.unique(longdata.team)
teams.sort()
print(teams)

['ARI' 'ATL' 'BAL' 'BUF' 'CAR' 'CHI' 'CIN' 'CLE' 'DAL' 'DEN' 'DET' 'GB'
 'HOU' 'IND' 'JAX' 'KC' 'LAC' 'LAR' 'MIA' 'MIN' 'NE' 'NO' 'NYG' 'NYJ'
 'OAK' 'PHI' 'PIT' 'SEA' 'SF' 'TB' 'TEN' 'WSH']


In [52]:
blocked_teams = ['BAL','DAL','PHI','NYG','CIN']
week_start = 6 #inclusive
week_end = 15 #inclusive
wts = []
selects = []
var_mappings = []

week_range = range(week_start,week_end + 1)

for wk in week_range:
    for t in teams:
        try:
            var = longdata[(longdata.week==wk) & (longdata.team==t)]['wp'].values[0]
        except IndexError:
            var = -1000

        wts.append(var)
        var_mappings.append([wk,t,var])

weights = cp.Constant(wts)
selects = cp.Variable(len(teams)*len(week_range), boolean=True)
var_mappings = pd.DataFrame(var_mappings, columns=['week','team','prob'])

In [53]:
#Create constraints, setting blocked teams == zero and included teams to <= 1
constraints = []

In [54]:
#Weekly constraints
for e, wk in enumerate(week_range):
    indices = list(range(32 * e, 32 * (e+1)))
    constraints.append(sum(selects[indices]) == 1)               

In [55]:
#Team constraints
for e,t in enumerate(teams):
    indices = [x + e for x in np.multiply(32,range(len(week_range)))]
    
    if t in blocked_teams:
        constraints.append(sum(selects[indices]) == 0)
    else:
        constraints.append(sum(selects[indices]) <= 1)

In [36]:
obj = cp.Maximize(np.multiply(weights,selects))

prob = cp.Problem(obj,constraints)

prob.solve()

results = pd.DataFrame(columns=['week','team','prob'])

for e,variable in enumerate(prob.variables()[0]):
    if np.round(variable.value)==1:
        results = results.append(var_mappings.iloc[e])
        
results

,week,team,prob
20,6,NE,0.891120
35,7,BUF,0.801363
83,8,MIN,0.865656
123,9,SEA,0.754693
149,10,NO,0.876804
188,11,SF,0.847265
197,12,CHI,0.819665
228,13,CAR,0.809517
267,14,GB,0.875604
303,15,KC,0.789868


In [66]:
results.to_csv('results.csv',index=False)

In [ ]:
def optimize_season(week_start = 1, week_end = 17, blocked_teams = [], ):
    
    
    blocked_teams = ['BAL','DAL','PHI','NYG','CIN']
    week_start = 6 #inclusive
    week_end = 15 #inclusive
    wts = []
    selects = []
    var_mappings = []

    week_range = range(week_start,week_end + 1)

    for wk in week_range:
        for t in teams:
            try:
                var = longdata[(longdata.week==wk) & (longdata.team==t)]['wp'].values[0]
            except IndexError:
                var = -1000

            wts.append(var)
            var_mappings.append([wk,t,var])

    weights = cp.Constant(wts)
    selects = cp.Variable(len(teams)*len(week_range), boolean=True)
    var_mappings = pd.DataFrame(var_mappings, columns=['week','team','prob'])
    
    #Create constraints, setting blocked teams == zero and included teams to <= 1
    constraints = []
    
    #Weekly constraints
    for e, wk in enumerate(week_range):
        indices = list(range(32 * e, 32 * (e+1)))
        constraints.append(sum(selects[indices]) == 1)               